In [7]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timedelta

In [2]:
mt5.initialize()

True

In [3]:
def get_historical_data(from_date, to_date):
    history_orders = mt5.history_orders_get(from_date, to_date)
    history_deals = mt5.history_deals_get(from_date, to_date)

    if history_orders is None:
        st.error(f"No history orders, error code={mt5.last_error()}")
        return None, None

    return history_orders, history_deals

In [8]:
default_start_date = datetime(2020, 1, 1)
default_end_date = datetime.now()

history_orders, history_deals = get_historical_data(default_start_date, default_end_date)

In [9]:
history_orders

(TradeOrder(ticket=283387539, time_setup=1754435042, time_setup_msc=1754435042579, time_done=1754435121, time_done_msc=1754435121845, time_expiration=1754352000, type=2, type_time=1, type_filling=2, state=4, magic=0, position_id=283387539, position_by_id=0, reason=3, volume_initial=0.05, volume_current=0.0, price_open=23029.06, sl=23014.63, tp=23072.35, price_current=23028.94, price_stoplimit=0.0, symbol='US100.cash', comment='metafvg_nas', external_id='283387539-O'),
 TradeOrder(ticket=283387974, time_setup=1754435179, time_setup_msc=1754435179614, time_done=1754435179, time_done_msc=1754435179745, time_expiration=0, type=1, type_time=0, type_filling=1, state=4, magic=0, position_id=283387539, position_by_id=0, reason=4, volume_initial=0.05, volume_current=0.0, price_open=23014.63, sl=0.0, tp=0.0, price_current=23014.63, price_stoplimit=0.0, symbol='US100.cash', comment='[sl 23014.63]', external_id='283387974-O'),
 TradeOrder(ticket=283388782, time_setup=1754435340, time_setup_msc=175

In [10]:
df_deals = pd.DataFrame(list(history_deals), columns=history_deals[0]._asdict().keys())

# Filter deals
df_deals_opens = df_deals[df_deals["comment"].str.contains("meta")]
df_deals_closes = df_deals[
    (df_deals["comment"].str.contains("sl")) |
    (df_deals["comment"].str.contains("tp")) |
    (df_deals["comment"].str.contains("Close"))
    ]

# Merge open and close deals
merged_deals = df_deals_closes.merge(
    df_deals_opens, on="position_id", suffixes=("_close", "_open")
)

# Process time columns
merged_deals["time_open"] = pd.to_datetime(merged_deals["time_open"], unit='s')
if "time_close" in merged_deals.columns:
    merged_deals["time_close"] = pd.to_datetime(merged_deals["time_close"], unit='s')

# Calculate strategy metrics - now using symbol_open and comment_open for grouping
strategy_metrics_df = merged_deals[
    ["profit_open", "profit_close", "comment_open", "symbol_open", "time_open"]].copy()

In [13]:
merged_deals.to_pickle("historical_merged_deals.pkl")

In [16]:
pd.concat([merged_deals, merged_deals]).drop_duplicates(subset=["symbol_open", "time_open", "position_id"], keep="first")

,ticket_close,order_close,time_close,time_msc_close,type_close,entry_close,magic_close,position_id,reason_close,volume_close,...,reason_open,volume_open,price_open,commission_open,swap_open,profit_open,fee_open,symbol_open,comment_open,external_id_open
0,271281178,283387974,2025-08-05 23:06:19,1754435179745,1,1,0,283387539,4,0.05,...,3,0.05,23028.84,0.0,0.0,0.0,0.0,US100.cash,metafvg_nas,247455358
1,271284561,283393930,2025-08-05 23:23:40,1754436220985,1,1,0,283389393,4,0.05,...,3,0.05,23028.84,0.0,0.0,0.0,0.0,US100.cash,metafvg_nas,247457597
2,271284786,283394153,2025-08-05 23:24:46,1754436286331,1,1,0,283388782,4,0.05,...,3,0.05,23021.14,0.0,0.0,0.0,0.0,US100.cash,metafvg_nas,247459482
3,271310937,283420631,2025-08-06 02:23:23,1754447003733,1,1,0,283419837,4,0.05,...,3,0.05,22982.05,0.0,0.0,0.0,0.0,US100.cash,metafvg_nas,247490649
4,271310939,283420632,2025-08-06 02:23:23,1754447003741,1,1,0,283420096,4,0.05,...,3,0.05,22982.05,0.0,0.0,0.0,0.0,US100.cash,metafvg_nas,247490650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,281231909,293635829,2025-08-22 23:14:08,1755904448253,1,1,234000,293599662,3,0.01,...,3,0.01,23538.65,0.0,0.0,0.0,0.0,US100.cash,metaga_i2,259237212
876,281231910,293635830,2025-08-22 23:14:08,1755904448406,1,1,234000,293602642,3,0.01,...,3,0.01,23523.75,0.0,0.0,0.0,0.0,US100.cash,metaga_i2,259240783
877,281231911,293635831,2025-08-22 23:14:08,1755904448568,1,1,234000,293603003,3,0.01,...,3,0.01,23518.75,0.0,0.0,0.0,0.0,US100.cash,metaga_i2,259241238
878,281231912,293635832,2025-08-22 23:14:08,1755904448718,1,1,234000,293603281,3,0.01,...,3,0.01,23519.65,0.0,0.0,0.0,0.0,US100.cash,metaga_i2,259241596


In [17]:
pd.read_pickle("historical_merged_deals.pkl")

,ticket_close,order_close,time_close,time_msc_close,type_close,entry_close,magic_close,position_id,reason_close,volume_close,...,reason_open,volume_open,price_open,commission_open,swap_open,profit_open,fee_open,symbol_open,comment_open,external_id_open
0,271281178,283387974,2025-08-05 23:06:19,1754435179745,1,1,0,283387539,4,0.05,...,3,0.05,23028.84,0.0,0.0,0.0,0.0,US100.cash,metafvg_nas,247455358
1,271284561,283393930,2025-08-05 23:23:40,1754436220985,1,1,0,283389393,4,0.05,...,3,0.05,23028.84,0.0,0.0,0.0,0.0,US100.cash,metafvg_nas,247457597
2,271284786,283394153,2025-08-05 23:24:46,1754436286331,1,1,0,283388782,4,0.05,...,3,0.05,23021.14,0.0,0.0,0.0,0.0,US100.cash,metafvg_nas,247459482
3,271310937,283420631,2025-08-06 02:23:23,1754447003733,1,1,0,283419837,4,0.05,...,3,0.05,22982.05,0.0,0.0,0.0,0.0,US100.cash,metafvg_nas,247490649
4,271310939,283420632,2025-08-06 02:23:23,1754447003741,1,1,0,283420096,4,0.05,...,3,0.05,22982.05,0.0,0.0,0.0,0.0,US100.cash,metafvg_nas,247490650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,281231909,293635829,2025-08-22 23:14:08,1755904448253,1,1,234000,293599662,3,0.01,...,3,0.01,23538.65,0.0,0.0,0.0,0.0,US100.cash,metaga_i2,259237212
876,281231910,293635830,2025-08-22 23:14:08,1755904448406,1,1,234000,293602642,3,0.01,...,3,0.01,23523.75,0.0,0.0,0.0,0.0,US100.cash,metaga_i2,259240783
877,281231911,293635831,2025-08-22 23:14:08,1755904448568,1,1,234000,293603003,3,0.01,...,3,0.01,23518.75,0.0,0.0,0.0,0.0,US100.cash,metaga_i2,259241238
878,281231912,293635832,2025-08-22 23:14:08,1755904448718,1,1,234000,293603281,3,0.01,...,3,0.01,23519.65,0.0,0.0,0.0,0.0,US100.cash,metaga_i2,259241596
